# PyKale Tutorial: Domain Adaptation on Digits with Lightning

| [Open in Colab](https://colab.research.google.com/github/pykale/pykale/blob/main/examples/digits_dann_lightn/tutorial.ipynb) (click `Runtime-->Run all (Ctrl+F9`) | [Launch Binder](https://mybinder.org/v2/gh/pykale/pykale/HEAD?filepath=examples%2Fdigits_dann_lightn%2Ftutorial.ipynb) (click `Cell-->Run All`) |

This tutorial is constructed based on the `digits_dann_lightn` example `main.py`, which is in turn refactored from the [ADA: (Yet) Another Domain Adaptation library](https://github.com/criteo-research/pytorch-ada).

It has been put together to run interactively on online hosting platforms including [Google Colab](https://colab.research.google.com) or [myBinder](https://mybinder.org), but can also be downloaded and run locally. Follow the [PyKale installation instructions](https://pykale.readthedocs.io/en/latest/installation.html) for this.

[Domain Adaptation](https://en.wikipedia.org/wiki/Domain_adaptation) takes a model trained and evaluated on one set of data (the source) and adapts it to another (the target). In this tutorial, a model is trained on one Digits Dataset (source) and adapted to another (target).

## Setup

The first few blocks of code are necessary to set up the notebook execution environment and import the required modules, including PyKale.

This checks if the notebook is running on Google Colab and installs required packages.

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !pip uninstall --yes imgaug && pip uninstall --yes albumentations && pip install git+https://github.com/aleju/imgaug.git
    !pip install git+https://github.com/pykale/pykale.git

    !git clone https://github.com/pykale/pykale.git
    %cd pykale/examples/digits_dann_lightn
else:
    print('Not running on CoLab')

This imports required modules.

In [1]:
import logging
import os

from config import get_cfg_defaults
import numpy as np
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torch.utils.data import SequentialSampler
import torchvision

from model import get_model
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint

from kale.loaddata.digits_access import DigitDataset
from kale.loaddata.multi_domain import MultiDomainDatasets
from kale.utils.seed import set_seed

ImportError: DLL load failed while importing qhull: 找不到指定的模块。

## Configuration

In this tutorial we provide a [default configuration for domain adaptation problems](https://github.com/pykale/pykale/blob/main/examples/digits_dann_lightn/config.py), which is tailored using a [`.yaml` file for the specific application in this tutorial](https://github.com/pykale/pykale/blob/main/examples/digits_dann_lightn/configs/TUTORIAL.yaml).

If GPUs are to be used at runtime, this is specified using a separate variable. If you are running this tutorial on Google Colab, or on a machine with GPU support, you might [set this](https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html#trainer-class-api) to make use of GPU acceleration. (On Google Colab click Runtime->Manage Sessions and select GPU, then change to `gpus = 1`).

The configuration is summarized below the following cell.

In [ ]:
cfg_path = "./configs/tutorial.yaml" # Path to `.yaml` config file
gpus = None # GPU settings

cfg = get_cfg_defaults()
cfg.merge_from_file(cfg_path)
cfg.freeze()
print(cfg)

## Select Datasets

Source and target datasets are specified using `DigitDataset.get_source_target` from values in the configuration (`cfg`) above. In this tutorial, we specify a subset of classes (1, 3 and 8) to make training and testing quicker.

In [ ]:
source, target, num_channels = DigitDataset.get_source_target(
    DigitDataset(cfg.DATASET.SOURCE.upper()), DigitDataset(cfg.DATASET.TARGET.upper()), cfg.DATASET.ROOT
)

class_subset = [1, 3, 8]

dataset = MultiDomainDatasets(
    source,
    target,
    config_weight_type=cfg.DATASET.WEIGHT_TYPE,
    config_size_type=cfg.DATASET.SIZE_TYPE,
    val_split_ratio=cfg.DATASET.VAL_SPLIT_RATIO,
    class_ids=class_subset,
)

## Set Seed

Some algorithms used in model training require generation of pseudo-random numbers. Setting the seed from which these are generated ensures reproducibility.

In [ ]:
seed = cfg.SOLVER.SEED
# seed_everything in pytorch_lightning did not set torch.backends.cudnn
set_seed(seed)

## Setup Model

Here, we use the previously defined configuration and dataset to set up the model we will subsequently train.

In [ ]:
%time model, train_params = get_model(cfg, dataset, num_channels)

Output reports on data file use.

## Setup Logger

A Tensorboard logger is used to store output generated during model training. This information can be used to assess the effectiveness of the training and to identify problems. The output model is also stored in `cfg.OUTPUT.TB_DIR`.

In [ ]:
tb_logger = pl_loggers.TensorBoardLogger(cfg.OUTPUT.TB_DIR, name="seed{}".format(seed))

## Setup Checkpoint

A ModelCheckpoint is used to save the model periodically by monitoring a quantity.

In [ ]:
checkpoint_callback = ModelCheckpoint(filename="{epoch}-{step}-{val_loss:.4f}", monitor="val_loss", mode="min",)

## Setup Trainer

A trainer object is used to determine and store model parameters. Here, one is configured with information on how a model should be trained, and what hardware will be used.

In [ ]:
trainer = pl.Trainer(
    progress_bar_refresh_rate=cfg.OUTPUT.PB_FRESH,  # in steps
    min_epochs=cfg.SOLVER.MIN_EPOCHS,
    max_epochs=cfg.SOLVER.MAX_EPOCHS,
    callbacks=[checkpoint_callback],
    logger=False,
    gpus=gpus)

Output reports on available GPU and TPU resources.

## Train Model

Optimize model parameters using the trainer.

In [ ]:
%time trainer.fit(model)

## Test Optimized Model

Check performance of model optimized with training data against test data which was not used in training.

In [ ]:
# test scores
%time trainer.test()

Outputs are defined as:

* 'Te_domain_acc': Accuracy on classifying the domain (source or target) from which data came.
* 'Te_source_acc': Accuracy on test data drawn from the source dataset.
* 'Te_target_acc': Accuracy on test data drawn from the target dataset.
* 'test_loss': Loss function value on the test data.